# 📊 Full Database Embeddings Analysis & Visualization

This notebook performs comprehensive analysis of the expanded IFC publication database using embeddings and creates various visualizations to understand the research landscape.

## What This Notebook Does:
- Loads the complete expanded database (IFC + PubMed articles)
- Generates embeddings for all publications using Google Gemini
- Creates 2D/3D visualizations using t-SNE and UMAP
- Performs clustering analysis
- Builds similarity networks
- Compares IFC vs PubMed article distributions
- Identifies research themes and trends

## 1. Setup and Data Loading

In [ ]:
# Core imports
import json
import pandas as pd
import numpy as np
from datetime import datetime
import os
from pathlib import Path

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ML and embeddings
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
import umap

# Google Gemini for embeddings
from google import genai
from google.genai import types
from tqdm.auto import tqdm

# Network analysis
import networkx as nx

print("✅ All imports successful!")

In [ ]:
# Configuration
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
tqdm.pandas()

# Set random seed for reproducibility
np.random.seed(42)

print("📊 Visualization settings configured")

In [ ]:
# Load the expanded database
database_path = 'data/processed/expanded_ifc_publications.json'

try:
    with open(database_path, 'r', encoding='utf-8') as f:
        database_data = json.load(f)
    
    publications = database_data['publications']
    metadata = database_data['metadata']
    
    print(f"📚 Loaded {len(publications)} publications from expanded database")
    print(f"📊 Database metadata:")
    print(f"   • Total publications: {metadata['total_publications']}")
    print(f"   • Last updated: {metadata['last_updated']}")
    print(f"   • Original IFC articles: {metadata['sources']['original_ifc_scraping']}")
    print(f"   • New PubMed articles: {metadata['sources']['pubmed_filtered_search']}")
    print(f"   • New articles added: {metadata['new_articles_added']}")
    
except FileNotFoundError:
    print("❌ Expanded database not found. Please run the database expansion notebook first.")
    print("Expected file: data/processed/expanded_ifc_publications.json")
    raise

In [ ]:
# Convert to DataFrame for easier analysis
df = pd.DataFrame(publications)

# Add source column to distinguish IFC vs PubMed articles
df['source_type'] = df['metadata'].apply(
    lambda x: 'PubMed' if x.get('source') == 'PubMed_filtered_search' else 'IFC'
)

# Create text for embedding (title + abstract)
df['embedding_text'] = df['title'].fillna('') + ' ' + df['abstract'].fillna('')
df['embedding_text'] = df['embedding_text'].str.strip()

# Basic statistics
print(f"📊 Dataset Overview:")
print(f"   • Total articles: {len(df)}")
print(f"   • IFC articles: {len(df[df['source_type'] == 'IFC'])}")
print(f"   • PubMed articles: {len(df[df['source_type'] == 'PubMed'])}")
print(f"   • Articles with abstracts: {len(df[df['abstract'].notna()])}")
print(f"   • Year range: {df['year'].min()} - {df['year'].max()}")

df.head()

## 2. Google Gemini Setup for Embeddings

In [ ]:
# Setup Gemini client
# Make sure you have GEMINI_API_KEY in your environment
import os

try:
    api_key = os.getenv('GEMINI_API_KEY')
    if not api_key:
        print("⚠️  GEMINI_API_KEY not found in environment variables.")
        print("Please set it with: export GEMINI_API_KEY='your-api-key'")
        # For testing purposes, you can uncomment and set it directly:
        # api_key = 'your-api-key-here'
        raise ValueError("API key required")
    
    client = genai.Client(api_key=api_key)
    print("✅ Gemini client initialized successfully")
    
    # List available embedding models
    print("\n📋 Available embedding models:")
    for m in client.models.list():
        if 'embedContent' in m.supported_actions:
            print(f"   • {m.name}")
            
except Exception as e:
    print(f"❌ Error setting up Gemini client: {e}")
    print("\n💡 Alternative: You can load pre-computed embeddings if available")

In [ ]:
# Select embedding model
MODEL_ID = "gemini-embedding-001"
print(f"🎯 Using embedding model: {MODEL_ID}")

## 3. Generate Embeddings

In [ ]:
def make_embed_text_fn(model_id, client):
    """Create embedding function for batch processing"""
    
    def embed_fn(text: str) -> list[float]:
        """Generate embedding for a single text"""
        try:
            result = client.models.embed_content(
                model=model_id,
                contents=text,
                config=types.EmbedContentConfig(task_type="CLUSTERING")
            )
            return np.array(result.embeddings[0].values)
        except Exception as e:
            print(f"Error embedding text: {str(e)[:100]}...")
            # Return zero vector as fallback
            return np.zeros(768)  # Default embedding size for gemini-embedding-001
    
    return embed_fn

# Create embedding function
embed_fn = make_embed_text_fn(MODEL_ID, client)
print("🔧 Embedding function created")

In [ ]:
# Check if embeddings already exist
embeddings_file = 'data/processed/publication_embeddings.npy'
embeddings_meta_file = 'data/processed/publication_embeddings_meta.json'

if os.path.exists(embeddings_file) and os.path.exists(embeddings_meta_file):
    print("📁 Found existing embeddings, loading...")
    embeddings = np.load(embeddings_file)
    with open(embeddings_meta_file, 'r') as f:
        emb_meta = json.load(f)
    print(f"✅ Loaded {embeddings.shape[0]} embeddings of dimension {embeddings.shape[1]}")
    
else:
    print("🔄 Generating embeddings for all publications...")
    print(f"📊 Processing {len(df)} publications")
    
    # Filter out empty texts
    df_embed = df[df['embedding_text'].str.len() > 10].copy()
    print(f"📋 {len(df_embed)} publications have sufficient text for embedding")
    
    # Generate embeddings with progress bar
    embeddings_list = []
    failed_indices = []
    
    for idx, text in tqdm(df_embed['embedding_text'].items(), 
                         desc="Generating embeddings", 
                         total=len(df_embed)):
        try:
            embedding = embed_fn(text[:8000])  # Limit text length
            embeddings_list.append(embedding)
        except Exception as e:
            print(f"Failed to embed article {idx}: {e}")
            failed_indices.append(idx)
            embeddings_list.append(np.zeros(768))  # Zero vector fallback
    
    embeddings = np.array(embeddings_list)
    
    # Save embeddings for future use
    os.makedirs('data/processed', exist_ok=True)
    np.save(embeddings_file, embeddings)
    
    emb_meta = {
        'model': MODEL_ID,
        'generated_at': datetime.now().isoformat(),
        'total_embeddings': len(embeddings),
        'embedding_dimension': embeddings.shape[1],
        'failed_indices': failed_indices
    }
    
    with open(embeddings_meta_file, 'w') as f:
        json.dump(emb_meta, f, indent=2)
    
    print(f"✅ Generated and saved {len(embeddings)} embeddings")
    print(f"💾 Saved to: {embeddings_file}")

# Add embeddings to dataframe
if len(embeddings) == len(df):
    df['embedding'] = [emb for emb in embeddings]
else:
    # Handle case where we filtered some articles
    print(f"⚠️  Embedding count mismatch. Adjusting...")
    # Add embeddings only to articles that have sufficient text
    df_embed = df[df['embedding_text'].str.len() > 10].copy()
    df = df_embed.copy()
    df['embedding'] = [emb for emb in embeddings]

print(f"📊 Final dataset: {len(df)} publications with embeddings")

## 4. Dimensionality Reduction & Visualization

In [ ]:
# Prepare embeddings matrix
embeddings_matrix = np.stack(df['embedding'].values)
print(f"📊 Embeddings matrix shape: {embeddings_matrix.shape}")

# t-SNE visualization
print("🔄 Computing t-SNE...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
tsne_results = tsne.fit_transform(embeddings_matrix)

df['tsne_x'] = tsne_results[:, 0]
df['tsne_y'] = tsne_results[:, 1]

print("✅ t-SNE completed")

In [ ]:
# UMAP visualization
print("🔄 Computing UMAP...")
umap_reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
umap_results = umap_reducer.fit_transform(embeddings_matrix)

df['umap_x'] = umap_results[:, 0]
df['umap_y'] = umap_results[:, 1]

print("✅ UMAP completed")

In [ ]:
# Create interactive t-SNE plot
fig = px.scatter(
    df, 
    x='tsne_x', 
    y='tsne_y', 
    color='source_type',
    hover_data={'title': True, 'year': True, 'journal': True},
    title='t-SNE Visualization of Publication Embeddings',
    width=800, 
    height=600
)

fig.update_layout(
    title_x=0.5,
    showlegend=True,
    xaxis_title="t-SNE Dimension 1",
    yaxis_title="t-SNE Dimension 2"
)

fig.show()

print("📊 Interactive t-SNE plot generated")

In [ ]:
# Create interactive UMAP plot
fig = px.scatter(
    df, 
    x='umap_x', 
    y='umap_y', 
    color='source_type',
    hover_data={'title': True, 'year': True, 'journal': True},
    title='UMAP Visualization of Publication Embeddings',
    width=800, 
    height=600
)

fig.update_layout(
    title_x=0.5,
    showlegend=True,
    xaxis_title="UMAP Dimension 1",
    yaxis_title="UMAP Dimension 2"
)

fig.show()

print("📊 Interactive UMAP plot generated")

## 5. Clustering Analysis

In [ ]:
# K-means clustering
print("🔄 Performing K-means clustering...")

# Test different numbers of clusters
k_range = range(3, 12)
inertias = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(embeddings_matrix)
    inertias.append(kmeans.inertia_)

# Plot elbow curve
plt.figure(figsize=(10, 6))
plt.plot(k_range, inertias, 'bo-')
plt.title('K-means Elbow Method')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()

# Choose optimal k (you can adjust this)
optimal_k = 6
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(embeddings_matrix)

df['cluster'] = cluster_labels

print(f"✅ K-means clustering completed with {optimal_k} clusters")
print("📊 Cluster sizes:")
print(df['cluster'].value_counts().sort_index())

In [ ]:
# Plot clusters on t-SNE
fig = px.scatter(
    df, 
    x='tsne_x', 
    y='tsne_y', 
    color='cluster',
    symbol='source_type',
    hover_data={'title': True, 'year': True, 'journal': True},
    title=f't-SNE Visualization with {optimal_k} Clusters',
    width=900, 
    height=700
)

fig.update_layout(title_x=0.5)
fig.show()

print("📊 Cluster visualization generated")

## 6. Source Comparison Analysis

In [ ]:
# Compare IFC vs PubMed distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Cluster distribution by source
cluster_source = pd.crosstab(df['cluster'], df['source_type'], normalize='columns')
cluster_source.plot(kind='bar', ax=axes[0,0], title='Cluster Distribution by Source')
axes[0,0].set_ylabel('Proportion')
axes[0,0].legend(title='Source')

# Year distribution by source
df.groupby(['source_type', 'year']).size().unstack(fill_value=0).T.plot(
    kind='line', ax=axes[0,1], title='Publication Timeline by Source'
)
axes[0,1].set_ylabel('Number of Publications')
axes[0,1].legend(title='Source')

# Abstract length distribution
df['abstract_length'] = df['abstract'].fillna('').str.len()
df.boxplot(column='abstract_length', by='source_type', ax=axes[1,0])
axes[1,0].set_title('Abstract Length Distribution by Source')
axes[1,0].set_xlabel('Source Type')

# Cluster sizes
cluster_sizes = df['cluster'].value_counts().sort_index()
cluster_sizes.plot(kind='bar', ax=axes[1,1], title='Cluster Sizes')
axes[1,1].set_ylabel('Number of Publications')
axes[1,1].set_xlabel('Cluster ID')

plt.tight_layout()
plt.show()

print("📊 Source comparison analysis completed")

## 7. Similarity Analysis

In [ ]:
# Calculate pairwise similarities (sample for performance)
sample_size = min(200, len(df))  # Limit for computational efficiency
sample_indices = np.random.choice(len(df), sample_size, replace=False)
sample_df = df.iloc[sample_indices].copy()
sample_embeddings = embeddings_matrix[sample_indices]

print(f"📊 Computing similarities for {sample_size} sample publications...")

# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(sample_embeddings)

# Plot similarity heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(similarity_matrix, 
            cmap='viridis', 
            center=0.5,
            square=True,
            cbar_kws={"shrink": .8})
plt.title(f'Cosine Similarity Heatmap ({sample_size} publications)')
plt.show()

print("📊 Similarity heatmap generated")

In [ ]:
# Find most similar publications
def find_most_similar_publications(df, embeddings, query_idx, top_k=5):
    """Find most similar publications to a given publication"""
    query_embedding = embeddings[query_idx].reshape(1, -1)
    similarities = cosine_similarity(query_embedding, embeddings)[0]
    
    # Get top k most similar (excluding the query itself)
    top_indices = np.argsort(similarities)[::-1][1:top_k+1]
    
    results = []
    for idx in top_indices:
        results.append({
            'index': idx,
            'similarity': similarities[idx],
            'title': df.iloc[idx]['title'],
            'year': df.iloc[idx]['year'],
            'source': df.iloc[idx]['source_type']
        })
    
    return results

# Example: Find similar publications to the first article
query_idx = 0
similar_pubs = find_most_similar_publications(df, embeddings_matrix, query_idx)

print(f"🔍 Publications most similar to: '{df.iloc[query_idx]['title']}'")
print("=" * 80)

for i, pub in enumerate(similar_pubs, 1):
    print(f"{i}. Similarity: {pub['similarity']:.3f} | {pub['source']} | {pub['year']}")
    print(f"   {pub['title'][:80]}...\n")

## 8. Insights & Summary

In [ ]:
# Analyze cluster characteristics
print("🔍 CLUSTER ANALYSIS")
print("=" * 50)

for cluster_id in sorted(df['cluster'].unique()):
    cluster_data = df[df['cluster'] == cluster_id]
    
    print(f"\n📊 Cluster {cluster_id} ({len(cluster_data)} publications):")
    print(f"   • IFC articles: {len(cluster_data[cluster_data['source_type'] == 'IFC'])}")
    print(f"   • PubMed articles: {len(cluster_data[cluster_data['source_type'] == 'PubMed'])}")
    
    # Year range
    years = cluster_data['year'].dropna()
    if len(years) > 0:
        print(f"   • Year range: {years.min()} - {years.max()}")
    
    # Top journals
    top_journals = cluster_data['journal'].value_counts().head(3)
    if len(top_journals) > 0:
        print(f"   • Top journals: {', '.join(top_journals.index[:3])}")
    
    # Sample titles
    sample_titles = cluster_data['title'].dropna().head(3)
    print(f"   • Sample titles:")
    for title in sample_titles:
        print(f"     - {title[:60]}...")

In [ ]:
# Final summary statistics
print("\n🎯 FINAL ANALYSIS SUMMARY")
print("=" * 50)

print(f"📚 Total Publications Analyzed: {len(df)}")
print(f"   • Original IFC publications: {len(df[df['source_type'] == 'IFC'])}")
print(f"   • New PubMed publications: {len(df[df['source_type'] == 'PubMed'])}")

print(f"\n🔬 Embeddings Analysis:")
print(f"   • Embedding model: {MODEL_ID}")
print(f"   • Embedding dimension: {embeddings_matrix.shape[1]}")
print(f"   • Clustering algorithm: K-means with {optimal_k} clusters")

print(f"\n📊 Research Coverage:")
year_range = df['year'].max() - df['year'].min()
print(f"   • Time span: {year_range} years ({df['year'].min()} - {df['year'].max()})")
print(f"   • Unique journals: {df['journal'].nunique()}")
print(f"   • Publications with abstracts: {len(df[df['abstract'].notna()])} ({len(df[df['abstract'].notna()])/len(df)*100:.1f}%)")

print(f"\n🎨 Visualizations Generated:")
print(f"   • t-SNE 2D embedding visualization")
print(f"   • UMAP 2D embedding visualization")
print(f"   • Cluster analysis plots")
print(f"   • Source comparison analysis")
print(f"   • Similarity heatmaps")

print(f"\n💾 Files Saved:")
print(f"   • {embeddings_file}")
print(f"   • {embeddings_meta_file}")

print("\n✅ Embeddings analysis completed successfully!")

## 🚀 Next Steps

After running this analysis, you can:

1. **Explore specific clusters** - Dive deep into publications within each cluster
2. **Topic modeling** - Use the embeddings for more sophisticated topic analysis
3. **Recommendation system** - Build a publication recommendation engine
4. **Time series analysis** - Analyze how research themes evolve over time
5. **Network analysis** - Create citation networks or collaboration graphs
6. **Podcast generation** - Use cluster information to generate themed podcasts

The embeddings and analysis results are saved and can be used in subsequent notebooks!